## Libraries

In [1]:
!pip install numpy
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install skll

import matplotlib.pyplot as plt

import sklearn.metrics as sklm
import skll.metrics as skllm
import copy
import json

import pandas as pd
import numpy as np

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Utils

In [2]:
PATH = "./"
ROOT_PATH = "../../"

In [3]:
def retrieve_data(flow_interval):
    path = "{0}dataset/dataset_flow_{1}.csv".format(ROOT_PATH, flow_interval)
    print(PATH)
    data = pd.read_csv(path, ';')
    
    data['Flow'].apply(int)
    data['AveSpeed'].apply(float)
    data['Density'].apply(float)
    data['Sunday'].apply(int)
    data['Monday'].apply(int)
    data['Tuesday'].apply(int)
    data['Wednesday'].apply(int)
    data['Thursday'].apply(int)
    data['Friday'].apply(int)
    data['Saturday'].apply(int)
      
    return data

In [4]:
def print_json (obj):
  print(json.dumps(obj, sort_keys=True, indent=4))

In [5]:
def store(obj, path, name):
  with open("{0}{1}/{2}.json".format(PATH, path, name), 'w') as json_file:
    json.dump(obj, json_file, sort_keys=True, indent=4)

## Testing

In [6]:
def plot_prediction (Y, Y_hat, title, alpha, gamma):
  """ Plot Prediction
  
  Plot the prediction (Flow x Time) of what was expected and what
  was predicted.
  """
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  name = f"{title}_alpha_d{alpha_dec}_gamma_d{gamma_dec})"
  path = f"plots/prediction/{name}"
    
  plt.plot(Y)
  plt.plot(Y_hat)
  plt.title(f"Predição do Modelo {title} (alpha {alpha}, gamma {gamma})")
  plt.ylabel('Fluxo')
  plt.xlabel('Tempo')
  plt.legend(['esperado', 'observado'], loc='upper left')
  plt.rcdefaults()

  plt.savefig(path + ".png", bbox_inches='tight')
  # plt.savefig(path + ".pdf")

  plt.close('all')

In [7]:
# Model Parameters
FLOW_INTERVAL = 150 # the interval size for each flow

# from 0.0 to 1.0 in intervals of 0.01
ALPHAS = np.arange(0.0, 1.0, 0.01).tolist() 
GAMMAS = np.arange(0.0, 1.0, 0.01).tolist()

In [8]:
def double_exponential_smoothing(X, alpha, gamma):
  global result_data
  
  name = "DES"
  expected, observed = [], []
  W = 1

  X_test = np.array_split(X, 10)
  F = X_test[8][-1]
  Lm1 = F
  bm1 = 0
  for x in X_test[9]:
    L = (alpha * x) + ( (1 - alpha) * (Lm1 + bm1) )
    b = ( gamma * (L - Lm1) ) + ( (1 - gamma) * bm1 )
    F1 = L + (b*W)
    
    expected.append(int(x))
    observed.append(float(F))
    F = F1
    Lm1 = L
    bm1 = b
  
  result_data['results']['RMSE'] = sklm.mean_squared_error(expected, observed, squared=False)
  result_data['results']['MAE'] = sklm.mean_absolute_error(expected, observed)
  result_data['results']['Kappa'] = skllm.kappa(expected, observed)
  result_data['results']['expected'] = expected
  result_data['results']['observed'] = observed
  alpha_dec = str(alpha - int(alpha)).split('.')[1]
  gamma_dec = str(gamma - int(gamma)).split('.')[1]
  #store(result_data, "results", f"{name}_alpha_d{alpha_dec}_gamma_d{gamma_dec}")

In [9]:
def eval_results(alpha, gamma):
  result = {
      "Alpha": alpha,
      "Gamma": gamma,
      "RMSE": result_data['results']['RMSE'],
      "MAE": result_data['results']['MAE'],
      "Kappa": result_data['results']['Kappa']
  }

  if result['RMSE'] < current_best["RMSE"]:
      best_rmse.append(result)
      current_best["RMSE"] = result['RMSE']
  
  if result['MAE'] < current_best["MAE"]:
      best_mae.append(result)
      current_best["MAE"] = result['MAE']

  if result['Kappa'] > current_best["Kappa"]:
      best_kappa.append(result)
      current_best["Kappa"] = result['Kappa']

In [10]:
global result_data
global current_best
global best_rmse
global best_mae
global best_kappa

data = retrieve_data(FLOW_INTERVAL)
sequence = np.array(data['Flow'])

best_rmse = []
best_mae = []
best_kappa = []

current_best = {
  "RMSE": 1000,
  "MAE": 1000,
  "Kappa": -1,
}

progress = -1
for alpha, gamma in [(x,y) for x in ALPHAS for y in GAMMAS]:
  result_data = {
    'results': {},
    'meta': {
      'FLOW_INTERVAL': FLOW_INTERVAL,
    }
  }
  
  double_exponential_smoothing(sequence, alpha, gamma)

  eval_results(alpha, gamma)
  
  if progress < alpha:
    print(f'\rProgress: {int(alpha*100) + 1}%', end="")
    progress = alpha
  
  #expected = result_data['results']['expected']
  #observed = result_data['results']['observed']
  #plot_prediction(expected, observed, "DES", alpha, gamma)

./
Progress: 100%

In [11]:
store(best_rmse, "results", "best_rmse")
store(best_mae, "results", "best_mae")
store(best_kappa, "results", "best_kappa")